In [1]:
import os
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import cvxpy as cvx

In [2]:
#Only 19 ticker in MMI constitutes after merge of DOW-DuPont
list_MMI = ['DWDP', 'CVX', 'WFC', 'MRK', 'XOM', 'DIS', 'JPM', 'KO', 'PG', 'JNJ', 'HPQ', 'MSFT', 
               'AXP', 'IBM', 'WMT', 'BA', 'MMM', 'MCD', 'GE']
N = len(list_MMI) #19
date0 = '2015-01-01'
date1 = '2019-01-01'

In [3]:
#load data or use cached data when force_refresh is false. 
force_refresh= False
file_ret = 'MMI_DailyReturns.csv'
file_mktcap = 'MMI_MktCap.csv'
if force_refresh:
    df_data = pdr.get_data_yahoo(list_MMI, date0, date1)
    df_price = df_data['Adj Close']
    df_ret = np.log(df_price).diff().dropna(how='all')
    df_ret.to_csv(file_ret, index=True)
    #use illustrative MktCap data. 
    ie_mktcap = [i+1 for i in xrange(13)]+[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]
    df_mktcap = pd.DataFrame(index=df_ret.index, columns=df_ret.columns)
    df_mktcap.iloc[0]=ie_mktcap
    df_mktcap = df_mktcap.fillna(method='ffill')
    df_mktcap.to_csv(file_mktcap, index=True)
else:
    df_ret = pd.read_csv(os.path.join('data',file_ret), index_col=0, parse_dates=True)
    df_mktcap = pd.read_csv(os.path.join('data',file_mktcap), index_col=0, parse_dates=True)
print('DateRange {t0} to {t1}'.format(t0=df_ret.index.min().date(), t1=df_ret.index.max().date()))
idx_MMI = df_ret.columns

DateRange 2015-01-05 to 2018-12-31


In [4]:
#alway use excess return, assuming fixed risk free rate for simplicity
srs_rf = pd.Series(0.01/252, index=df_ret.index) #daily risk free rate
df_ret_e = df_ret.apply(lambda x: x - srs_rf)
df_ret_m = df_ret_e.resample('M').sum()

#### Chapter 2 Risk
https://pdfs.semanticscholar.org/75cc/ed88c1199ba9e8607774b62f2af4c92f0875.pdf

------------
#### Notation:
* h - holding vector
    - h_a - holding solved by analytical solution
    - h_x - holding solved by optimization
* f - vector of expected excess return 
    - f_B - forcast benchmark return 
* mu - vector of expected excess return in CAPM.  f=mu if CAPM holds. 
* V - Covariance Matrix
* beta - vector of asset beta
* e - vector of ones. 
* a - attribute, can be e, beta, alpha, or factor loading

In [5]:
# calc covariance matrix, using monthly return 
df_cov = df_ret_m.cov()
df_mkt_w = (df_mktcap.T/df_mktcap.T.sum()).T
df_mkt = (df_mkt_w*df_ret_m).dropna(how='all').sum(axis=1).to_frame(name='MKT')
df_ret_fullmkt = pd.concat([df_mkt, df_ret_m], axis=1)

V = df_cov
V_inv = np.linalg.inv(V)
beta = (df_ret_fullmkt.cov() / df_mkt.var())['MKT'][1:]
e = np.ones(N)


#f = df_ret_m.mean()
np.random.seed = 123
#use randomg alphas
alpha = pd.Series(np.random.standard_normal(N), index=list_MMI)
f_MKT = 0.01
mu = beta*f_MKT
f = mu + alpha

--------------------
#### Characteristic Portfolios

In [6]:
a = e
name = 'Portfolio C (min variance of full invested portfolio)'
print(name)

h_a = V_inv.dot(a) / a.T.dot(V_inv).dot(a) 
var = h_a.T.dot(V).dot(h_a)
print("Analytical Solution, varince={v}".format(name=name, v=var))

#solve numerically
h_x = cvx.Variable(N)
objective = cvx.Minimize(cvx.quad_form(h_x, V))
prob = cvx.Problem(objective, constraints=[cvx.sum(h_x)==1])
solv = prob.solve(solver='CVXOPT')
print("Optimization Solution, varince={v}".format(name=name, v=solv))
print("variance diff {d}".format(d=solv - var))
print("Holding difference {d}".format(d = np.sum(np.abs(h_x.value - h_a))))

Portfolio C (min variance of full invested portfolio)
Analytical Solution, varince=0.000535476876684
Optimization Solution, varince=0.000535478020835
variance diff 1.14415142237e-09
Holding difference 1.09413346439e-14


In [7]:
a = beta
name = 'Portfolio B (min variance portfolio with beta 1)'
print(name)

h_a = V_inv.dot(a) / a.T.dot(V_inv).dot(a) 
var = h_a.T.dot(V).dot(h_a)
beta_a = h_a.dot(beta)
print("Analytical Solution, beta={v}".format(name=name, v=beta_a))

#solve numerically
h_x = cvx.Variable(N)
objective = cvx.Minimize(cvx.quad_form(h_x, V))
prob = cvx.Problem(objective, constraints=[cvx.sum(h_x*beta)==1])
solv = prob.solve(solver='CVXOPT')
beta_solv = h_x.value.dot(beta)
print("Optimization Solution, beta={v}".format(name=name, v=beta_solv))
print("beta diff {d}".format(d=beta_solv - beta_a))
print("Holding difference {d}".format(d = np.sum(np.abs(h_x.value - h_a))))

Portfolio B (min variance portfolio with beta 1)
Analytical Solution, beta=1.0
Optimization Solution, beta=1.0
beta diff 1.44328993201e-15
Holding difference 9.838484194e-15


In [8]:
a = f
name = 'Portfolio q, tangent portfolio, maximizing sharpe ratio'
print(name)

h_a = V_inv.dot(a) / a.T.dot(V_inv).dot(a) 
var = h_a.T.dot(V).dot(h_a)
#sr_a = a.T.dot(V_inv).dot(a)**0.5
sr_a=h_a.T.dot(f)/var**0.5
print("Analytical Solution, SR={v}".format(name=name, v=sr_a))

#solve numerically
h_x = cvx.Variable(N)
objective = cvx.Minimize(cvx.quad_form(h_x, V))
prob = cvx.Problem(objective, constraints=[cvx.sum(h_x*f)==1])
solv = prob.solve(solver='CVXOPT')
h_q_cvx = h_x.value
var_cvx = h_q_cvx.T.dot(V).dot(h_q_cvx)
sr_cvx = h_q_cvx.T.dot(f)/var**0.5
print("Optimization Solution, SR={v}".format(name=name, v=sr_cvx))
print("SR diff {d}".format(d=sr_cvx - sr_a))
print("Holding difference {d}".format(d = np.sum(np.abs(h_x.value - h_a))))

Portfolio q, tangent portfolio, maximizing sharpe ratio
Analytical Solution, SR=154.700999072
Optimization Solution, SR=154.700999072
SR diff 0.0
Holding difference 2.25167107182e-15


In [9]:
a = alpha
name = 'Portfolio A, alpha=1 using leverage. '
print(name)

h_a = V_inv.dot(a) / a.T.dot(V_inv).dot(a) 
var = h_a.T.dot(V).dot(h_a)
#sr_a = a.T.dot(V_inv).dot(a)**0.5
alpha_a=h_a.T.dot(alpha)
print("Analytical Solution, alpha={v}".format(name=name, v=alpha_a))

#solve numerically
h_x = cvx.Variable(N)
objective = cvx.Minimize(cvx.quad_form(h_x, V))
prob = cvx.Problem(objective, constraints=[cvx.sum(h_x*alpha)==1])
solv = prob.solve(solver='CVXOPT')
h_cvx = h_x.value
#var_cvx = h_q_cvx.T.dot(V).dot(h_q_cvx)
alpha_cvx=h_cvx.T.dot(alpha)
print("Optimization Solution, SR={v}".format(name=name, v=alpha_cvx))
print("alpha diff {d}".format(d=alpha_a - alpha_cvx))
print("Leverage {h}".format(h=h_cvx.sum()))
print("Holding difference {d}".format(d = np.sum(np.abs(alpha_a - alpha_cvx))))

Portfolio A, alpha=1 using leverage. 
Analytical Solution, alpha=1.0
Optimization Solution, SR=1.0
alpha diff 4.4408920985e-16
Leverage -0.0482321880186
Holding difference 4.4408920985e-16


In [10]:
a = alpha
name = 'Factor Portfolio, with unit exposure on attribute(i.e. alpha), sum of holding =0.  '
print(name)

# h_a = V_inv.dot(a) / a.T.dot(V_inv).dot(a) 
# var = h_a.T.dot(V).dot(h_a)
# #sr_a = a.T.dot(V_inv).dot(a)**0.5
# alpha_a=h_a.T.dot(alpha)
# print("Analytical Solution, alpha={v}".format(name=name, v=alpha_a))

#solve numerically
h_x = cvx.Variable(N)
objective = cvx.Minimize(cvx.quad_form(h_x, V))
prob = cvx.Problem(objective, constraints=[cvx.sum(h_x*alpha)==1, cvx.sum(h_x)==0])
solv = prob.solve(solver='CVXOPT')
h_cvx = h_x.value
#var_cvx = h_q_cvx.T.dot(V).dot(h_q_cvx)
alpha_cvx=h_cvx.T.dot(alpha)
print("Optimization Solution, factor exposure={v}".format(name=name, v=h_cvx.T.dot(a)))
print("Leverage {h}".format(h=h_cvx.sum()))
# print("Holding difference {d}".format(d = np.sum(np.abs(alpha_a - alpha_cvx))))

Factor Portfolio, with unit exposure on attribute(i.e. alpha), sum of holding =0.  
Optimization Solution, factor exposure=1.0
Leverage 1.11022302463e-16
